In [1]:
%%time

import sys
from urllib.request import urlretrieve
import os
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
import zipfile as zf

CPU times: user 1.67 s, sys: 2.66 s, total: 4.33 s
Wall time: 805 ms


In [2]:
%%time

# Download the data.
url = 'http://files.grouplens.org/datasets/movielens/'


def reporthook(blocknum, blocksize, totalsize):
    readsofar = blocknum * blocksize
    if totalsize > 0:
        percent = readsofar * 1e2 / totalsize
        s = "\r%5.1f%% %*d / %d" % (
            percent, len(str(totalsize)), readsofar, totalsize)
        sys.stderr.write(s)
        if readsofar >= totalsize: # near the end
            sys.stderr.write("\n")
    else: # total size is unknown
        sys.stderr.write("read %d\n" % (readsofar,))

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename,reporthook)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename


data_file = maybe_download('ml-20m.zip', 198702078)

100.0% 198696960 / 198702078

Found and verified ml-20m.zip
CPU times: user 3.57 s, sys: 1.46 s, total: 5.03 s
Wall time: 1min 44s


100.0% 198705152 / 198702078


In [3]:
%%time

with zf.ZipFile("ml-20m.zip", "r") as zip_ref:
    zip_ref.extractall()

CPU times: user 4.32 s, sys: 592 ms, total: 4.91 s
Wall time: 4.99 s


In [4]:
%%time

# file should look like
'''
userId,movieId,rating,timestamp
1,2,3.5,1112486027
1,29,3.5,1112484676
1,32,3.5,1112484819
1,47,3.5,1112484727
1,50,3.5,1112484580
1,112,3.5,1094785740
1,151,4.0,1094785734
1,223,4.0,1112485573
1,253,4.0,1112484940
'''

fileIN = open("ml-20m/ratings.csv", "r")
fileOUT = open("ratings.dat", "w")

map_items = {}
map_users = {}
n = 1
m = 1

f = fileIN.readlines()
f.pop(0)  # skip the first line

for line in f:
    token = line.strip().split(',')
    uid = token[0]
    iid = token[1]
    score = token[2]

    if uid in map_users:
        real_uid = map_users[uid]
    else:
        map_users[uid] = m
        real_uid = m
        m+=1

    if iid in map_items:
        real_iid = map_items[iid]
    else:
        map_items[iid] = n
        real_iid = n
        n+=1

    fileOUT.write(str(real_uid)+","+str(real_iid)+","+str(score)+"\n")
    
fileIN.close()
fileOUT.close()

CPU times: user 40.8 s, sys: 1.85 s, total: 42.6 s
Wall time: 41.7 s


In [5]:
%%time

m = 138_493
n = 26_744
nnz_train = 18_000_236
nnz_test = 2_000_027

data_filename = 'ratings.dat'

data = pd.read_csv(data_filename,dtype={0:'int32',1:'int32',2:'float32'},usecols=[0,1,2],header=None)

user = data[0].values
item = data[1].values
rating = data[2].values

CPU times: user 4.08 s, sys: 480 ms, total: 4.56 s
Wall time: 4.56 s


In [6]:
%%time

print(user)
print(item)
print(rating)
print("")
print(np.min(user))
print(np.min(item))
print(np.min(rating))
print("")
print(np.max(user))
print(np.max(item))
print(np.max(rating))
print("")
print(np.unique(user).size)
print(np.unique(item).size)
print(np.unique(rating).size)
print("")
print(user.size)

assert np.max(user) == m
assert np.max(item) == n
assert user.size == nnz_train + nnz_test

[     1      1      1 ... 138493 138493 138493]
[   1    2    3 ... 3951 1819 4011]
[3.5 3.5 3.5 ... 3.  5.  2.5]

1
1
0.5

138493
26744
5.0

138493
26744
10

20000263
CPU times: user 2.08 s, sys: 116 ms, total: 2.2 s
Wall time: 2.19 s


In [7]:
%%time

user_item = np.vstack((user, item))

user_item_train, user_item_test, rating_train, rating_test = train_test_split(user_item.T,
                                                                              rating,
                                                                              test_size=nnz_test,
                                                                              random_state=42)

CPU times: user 1.7 s, sys: 317 ms, total: 2.01 s
Wall time: 2.01 s


In [8]:
%%time

#1-based to 0-based
R_test_coo = sparse.coo_matrix((rating_test, (user_item_test[:, 0] - 1, user_item_test[:, 1] - 1)))
assert R_test_coo.nnz == nnz_test

outfile_test = open("test.txt", 'w')
for i in range(nnz_test):
    outfile_test.write(str(user_item_test[i, 0]) + " " + str(user_item_test[i, 1]) + " " + str(rating_test[i]) + "\n")
outfile_test.close()

CPU times: user 6.08 s, sys: 55.1 ms, total: 6.13 s
Wall time: 6.06 s


In [9]:
%%time

# for test data, we need COO format to calculate test RMSE

R_test_coo.data.astype(np.float32).tofile('R_test_coo.data.bin')
R_test_coo.row.tofile('R_test_coo.row.bin')
R_test_coo.col.tofile('R_test_coo.col.bin')

test_data = np.fromfile('R_test_coo.data.bin', dtype=np.float32)
test_row = np.fromfile('R_test_coo.row.bin', dtype=np.int32)
test_col = np.fromfile('R_test_coo.col.bin', dtype=np.int32)

CPU times: user 2.59 ms, sys: 32.6 ms, total: 35.2 ms
Wall time: 34.1 ms


In [10]:
%%time

print(R_test_coo.data)
print(R_test_coo.row)
print(R_test_coo.col)
print("")
print(test_data)
print(test_row)
print(test_col)

[3.5 2.  3.5 ... 2.5 3.5 5. ]
[122269  49017  89526 ...  29334 124294  73173]
[1154    2 2662 ... 1580  971  632]

[3.5 2.  3.5 ... 2.5 3.5 5. ]
[122269  49017  89526 ...  29334 124294  73173]
[1154    2 2662 ... 1580  971  632]
CPU times: user 2.48 ms, sys: 293 µs, total: 2.78 ms
Wall time: 2.22 ms


In [11]:
%%time

print(np.max(R_test_coo.data))
print(np.max(R_test_coo.row))
print(np.max(R_test_coo.col))
print("")
print(np.min(R_test_coo.data))
print(np.min(R_test_coo.row))
print(np.min(R_test_coo.col))
print("")
print(np.unique(user).size)
print(np.unique(R_test_coo.row).size)
print(np.unique(item).size)
print(np.unique(R_test_coo.col).size)

5.0
138492
26736

0.5
0
0

138493
135697
26744
17719
CPU times: user 1.53 s, sys: 112 ms, total: 1.64 s
Wall time: 1.63 s


In [12]:
%%time

#1-based to 0-based
R_train_coo = sparse.coo_matrix((rating_train, (user_item_train[:, 0] - 1, user_item_train[:, 1] - 1)))
assert R_train_coo.nnz == nnz_train

outfile_train = open("train.txt", 'w')
for i in range(nnz_train):
    outfile_train.write(str(user_item_train[i, 0]) + " " + str(user_item_train[i, 1]) + " " + str(rating_train[i]) + "\n")
outfile_train.close()

CPU times: user 55.7 s, sys: 679 ms, total: 56.4 s
Wall time: 55.8 s


In [13]:
%%time

# for training data, we need COO format to calculate training RMSE
# we need CSR format R when calculate X from \Theta
# we need CSC format of R when calculating \Theta from X
R_train_coo.data.astype(np.float32).tofile('R_train_coo.data.bin')
R_train_coo.row.tofile('R_train_coo.row.bin')
R_train_coo.col.tofile('R_train_coo.col.bin')

R_train_csr = R_train_coo.tocsr()
R_train_csc = R_train_coo.tocsc()

R_train_csr.data.astype(np.float32).tofile('R_train_csr.data.bin')
R_train_csr.indices.tofile('R_train_csr.indices.bin')
R_train_csr.indptr.tofile('R_train_csr.indptr.bin')
R_train_csc.data.astype(np.float32).tofile('R_train_csc.data.bin')
R_train_csc.indices.tofile('R_train_csc.indices.bin')
R_train_csc.indptr.tofile('R_train_csc.indptr.bin')

CPU times: user 3.42 s, sys: 477 ms, total: 3.9 s
Wall time: 3.99 s


In [14]:
%%time

train_data = np.fromfile('R_train_coo.data.bin', dtype=np.float32)
train_row = np.fromfile('R_train_coo.row.bin', dtype=np.int32)
train_col = np.fromfile('R_train_coo.col.bin', dtype=np.int32)

train_csc_data = np.fromfile('R_train_csc.data.bin', dtype=np.float32)
train_csc_indices = np.fromfile('R_train_csc.indices.bin', dtype=np.int32)
train_csc_indptr = np.fromfile('R_train_csc.indptr.bin', dtype=np.int32)

train_csr_data = np.fromfile('R_train_csr.data.bin', dtype=np.float32)
train_csr_indices = np.fromfile('R_train_csr.indices.bin', dtype=np.int32)
train_csr_indptr = np.fromfile('R_train_csr.indptr.bin', dtype=np.int32)

CPU times: user 2.37 ms, sys: 262 ms, total: 264 ms
Wall time: 263 ms


In [15]:
%%time

print(R_train_coo.data)
print(R_train_coo.row)
print(R_train_coo.col)
print("")
print(train_data)
print(train_row)
print(train_col)
print("")
print(R_train_csr.data)
print(R_train_csr.indices)
print(R_train_csr.indptr)
print("")
print(train_csr_data)
print(train_csr_indices)
print(train_csr_indptr)
print("")
print(R_train_csc.data)
print(R_train_csc.indices)
print(R_train_csc.indptr)
print("")
print(train_csc_data)
print(train_csc_indices)
print(train_csc_indptr)

[2.  3.5 1.5 ... 3.5 5.  3.5]
[123513  33254 101943 ...  15190  92010 111372]
[1600 7339 1747 ...  744  274  335]

[2.  3.5 1.5 ... 3.5 5.  3.5]
[123513  33254 101943 ...  15190  92010 111372]
[1600 7339 1747 ...  744  274  335]

[3.5 3.5 3.5 ... 4.5 5.  4.5]
[    0     3     4 ... 11011 11504 11614]
[       0      158      216 ... 17999818 17999891 18000236]

[3.5 3.5 3.5 ... 4.5 5.  4.5]
[    0     3     4 ... 11011 11504 11614]
[       0      158      216 ... 17999818 17999891 18000236]

[3.5 3.  3.  ... 4.5 4.5 4.5]
[     0      4     12 ... 138300 138405 138405]
[       0    20053    27714 ... 18000234 18000235 18000236]

[3.5 3.  3.  ... 4.5 4.5 4.5]
[     0      4     12 ... 138300 138405 138405]
[       0    20053    27714 ... 18000234 18000235 18000236]
CPU times: user 3.93 ms, sys: 0 ns, total: 3.93 ms
Wall time: 2.77 ms


In [16]:
%%time

print(np.max(R_train_coo.data))
print(np.max(R_train_coo.row))
print(np.max(R_train_coo.col))
print("")
print(np.min(R_train_coo.data))
print(np.min(R_train_coo.row))
print(np.min(R_train_coo.col))
print("")
print(np.unique(user).size)
print(np.unique(R_train_coo.row).size)
print(np.unique(item).size)
print(np.unique(R_train_coo.col).size)

5.0
138492
26743

0.5
0
0

138493
138493
26744
26325
CPU times: user 3.43 s, sys: 216 ms, total: 3.65 s
Wall time: 3.64 s


In [17]:
%%time
#NNZ by cols
print(len(R_train_csr.getnnz(axis=0)))
print(R_train_csr.getnnz(axis=0))
print("")
print(np.min(R_train_csr.getnnz(axis=0)))
print(np.max(R_train_csr.getnnz(axis=0)))
print("")
print(np.mean(R_train_csr.getnnz(axis=0)))

26744
[20053  7661 40434 ...     1     1     1]

0
60526

673.0569847442417
CPU times: user 239 ms, sys: 262 ms, total: 501 ms
Wall time: 495 ms


In [18]:
%%time
#NNZ by rows
print(len(R_train_csr.getnnz(axis=1)))
print(R_train_csr.getnnz(axis=1))
print("")
print(np.min(R_train_csr.getnnz(axis=1)))
print(np.max(R_train_csr.getnnz(axis=1)))
print("")
print(np.mean(R_train_csr.getnnz(axis=1)))

138493
[158  58 170 ...  18  73 345]

12
8315

129.97217187872312
CPU times: user 3.57 ms, sys: 126 µs, total: 3.7 ms
Wall time: 2.2 ms


In [19]:
%%time

print("write extra meta file")

outfile_meta = open("meta_modified_all", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n" + str(nnz_train) + "\n")
outfile_meta.write("""R_train_coo.data.bin
R_train_coo.row.bin
R_train_coo.col.bin
R_train_csr.indptr.bin
R_train_csr.indices.bin
R_train_csr.data.bin
R_train_csc.indptr.bin
R_train_csc.indices.bin
R_train_csc.data.bin
""")
outfile_meta.write(str(nnz_test) + "\n")
outfile_meta.write("""R_test_coo.data.bin
R_test_coo.row.bin
R_test_coo.col.bin
""")
outfile_meta.close()

write extra meta file
CPU times: user 735 µs, sys: 0 ns, total: 735 µs
Wall time: 493 µs


In [20]:
%%time

print("writing extra meta file")

outfile_meta = open("meta", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n")
outfile_meta.write(str(nnz_train) + " " + "train.txt\n")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")
outfile_meta.close()

writing extra meta file
CPU times: user 611 µs, sys: 46 µs, total: 657 µs
Wall time: 520 µs
